In [1]:
import pandas as pd
import numpy as np


ModuleNotFoundError: No module named 'calc'

In [4]:
# OHLCVデータをロード
df = pd.read_csv('bybit_btcusdt_20210101-20230330.csv', index_col=0)

In [6]:
# 短期および長期の単純移動平均 (SMA) を計算する関数
def calculate_sma(dataframe, column, window):
    return dataframe[column].rolling(window=window).mean()

# 短期および長期のSMAを計算して新たなカラムとして追加
short_term_window = 5  # 短期SMAのウィンドウ期間（例：5分）
long_term_window = 20  # 長期SMAのウィンドウ期間（例：20分）

df['SMA_short_term'] = calculate_sma(df, 'Close', short_term_window)
df['SMA_long_term'] = calculate_sma(df, 'Close', long_term_window)

# 結果を表示
print(df.tail())

                   Timestamp     Open     High      Low    Close  Volume  \
1179469  2023-03-30 16:49:00  28235.2  28235.3  28228.0  28229.1  18.223   
1179470  2023-03-30 16:50:00  28229.1  28241.9  28229.1  28241.0  21.137   
1179471  2023-03-30 16:51:00  28241.0  28258.1  28241.0  28258.0  32.133   
1179472  2023-03-30 16:52:00  28258.0  28258.6  28243.7  28243.7  64.725   
1179473  2023-03-30 16:53:00  28243.7  28268.9  28243.7  28268.9  33.430   

         SMA_short_term  SMA_long_term  
1179469        28238.04      28243.600  
1179470        28236.38      28242.670  
1179471        28240.86      28243.785  
1179472        28241.40      28245.790  
1179473        28248.14      28246.790  


In [8]:
# ボリンジャーバンドの上限、下限、中心線を計算する関数
def calculate_bollinger_bands(dataframe, column, window, num_std_dev):
    sma = dataframe[column].rolling(window=window).mean()
    std_dev = dataframe[column].rolling(window=window).std()
    
    upper_band = sma + (std_dev * num_std_dev)
    lower_band = sma - (std_dev * num_std_dev)
    
    return upper_band, lower_band, sma

# ボリンジャーバンドの上限、下限、中心線を計算して新たなカラムとして追加
window = 20  # ボリンジャーバンドのウィンドウ期間（例：20分）
num_std_dev = 2  # 標準偏差の倍数（通常は2）

df['BB_upper'], df['BB_lower'], df['BB_middle'] = calculate_bollinger_bands(df, 'Close', window, num_std_dev)

# 結果を表示
print(df.tail())

                   Timestamp     Open     High      Low    Close  Volume  \
1179469  2023-03-30 16:49:00  28235.2  28235.3  28228.0  28229.1  18.223   
1179470  2023-03-30 16:50:00  28229.1  28241.9  28229.1  28241.0  21.137   
1179471  2023-03-30 16:51:00  28241.0  28258.1  28241.0  28258.0  32.133   
1179472  2023-03-30 16:52:00  28258.0  28258.6  28243.7  28243.7  64.725   
1179473  2023-03-30 16:53:00  28243.7  28268.9  28243.7  28268.9  33.430   

         SMA_short_term  SMA_long_term      BB_upper      BB_lower  BB_middle  
1179469        28238.04      28243.600  28275.168868  28212.031132  28243.600  
1179470        28236.38      28242.670  28273.337249  28212.002751  28242.670  
1179471        28240.86      28243.785  28275.001879  28212.568121  28243.785  
1179472        28241.40      28245.790  28270.641630  28220.938370  28245.790  
1179473        28248.14      28246.790  28273.693401  28219.886599  28246.790  


In [10]:
# RSIを計算する関数
def calculate_rsi(dataframe, column, window):
    delta = dataframe[column].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window=window).mean()
    avg_loss = loss.rolling(window=window).mean()
    
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    return rsi

# RSIを計算して新たなカラムとして追加
window = 14  # RSIのウィンドウ期間（例：14分）

df['RSI'] = calculate_rsi(df, 'Close', window)

# 結果を表示
print(df.tail())

                   Timestamp     Open     High      Low    Close  Volume  \
1179469  2023-03-30 16:49:00  28235.2  28235.3  28228.0  28229.1  18.223   
1179470  2023-03-30 16:50:00  28229.1  28241.9  28229.1  28241.0  21.137   
1179471  2023-03-30 16:51:00  28241.0  28258.1  28241.0  28258.0  32.133   
1179472  2023-03-30 16:52:00  28258.0  28258.6  28243.7  28243.7  64.725   
1179473  2023-03-30 16:53:00  28243.7  28268.9  28243.7  28268.9  33.430   

         SMA_short_term  SMA_long_term      BB_upper      BB_lower  BB_middle  \
1179469        28238.04      28243.600  28275.168868  28212.031132  28243.600   
1179470        28236.38      28242.670  28273.337249  28212.002751  28242.670   
1179471        28240.86      28243.785  28275.001879  28212.568121  28243.785   
1179472        28241.40      28245.790  28270.641630  28220.938370  28245.790   
1179473        28248.14      28246.790  28273.693401  28219.886599  28246.790   

               RSI  
1179469  50.659898  
1179470  46.68

In [11]:
# MACDおよびシグナルラインを計算する関数
def calculate_macd(dataframe, column, short_window, long_window, signal_window):
    short_ema = dataframe[column].ewm(span=short_window).mean()
    long_ema = dataframe[column].ewm(span=long_window).mean()

    macd = short_ema - long_ema
    signal_line = macd.ewm(span=signal_window).mean()

    return macd, signal_line

# MACDおよびシグナルラインを計算して新たなカラムとして追加
short_window = 12  # 短期EMAのウィンドウ期間（例：12分）
long_window = 26   # 長期EMAのウィンドウ期間（例：26分）
signal_window = 9  # シグナルラインのウィンドウ期間（例：9分）

df['MACD'], df['Signal_line'] = calculate_macd(df, 'Close', short_window, long_window, signal_window)

# 結果を表示
print(df.tail())

                   Timestamp     Open     High      Low    Close  Volume  \
1179469  2023-03-30 16:49:00  28235.2  28235.3  28228.0  28229.1  18.223   
1179470  2023-03-30 16:50:00  28229.1  28241.9  28229.1  28241.0  21.137   
1179471  2023-03-30 16:51:00  28241.0  28258.1  28241.0  28258.0  32.133   
1179472  2023-03-30 16:52:00  28258.0  28258.6  28243.7  28243.7  64.725   
1179473  2023-03-30 16:53:00  28243.7  28268.9  28243.7  28268.9  33.430   

         SMA_short_term  SMA_long_term      BB_upper      BB_lower  BB_middle  \
1179469        28238.04      28243.600  28275.168868  28212.031132  28243.600   
1179470        28236.38      28242.670  28273.337249  28212.002751  28242.670   
1179471        28240.86      28243.785  28275.001879  28212.568121  28243.785   
1179472        28241.40      28245.790  28270.641630  28220.938370  28245.790   
1179473        28248.14      28246.790  28273.693401  28219.886599  28246.790   

               RSI       MACD  Signal_line  
1179469  50

In [14]:
# ヒストリカルボラティリティを計算する関数
def calculate_historical_volatility(dataframe, column, window):
    log_returns = np.log(dataframe[column] / dataframe[column].shift(1))
    historical_volatility = log_returns.rolling(window=window).std() * np.sqrt(252) * 100

    return historical_volatility

# ヒストリカルボラティリティを計算して新たなカラムとして追加
window = 14  # ヒストリカルボラティリティのウィンドウ期間（例：14分）

df['Historical_volatility'] = calculate_historical_volatility(df, 'Close', window)

# 結果を表示
print(df.tail())

                   Timestamp     Open     High      Low    Close  Volume  \
1179469  2023-03-30 16:49:00  28235.2  28235.3  28228.0  28229.1  18.223   
1179470  2023-03-30 16:50:00  28229.1  28241.9  28229.1  28241.0  21.137   
1179471  2023-03-30 16:51:00  28241.0  28258.1  28241.0  28258.0  32.133   
1179472  2023-03-30 16:52:00  28258.0  28258.6  28243.7  28243.7  64.725   
1179473  2023-03-30 16:53:00  28243.7  28268.9  28243.7  28268.9  33.430   

         SMA_short_term  SMA_long_term      BB_upper      BB_lower  BB_middle  \
1179469        28238.04      28243.600  28275.168868  28212.031132  28243.600   
1179470        28236.38      28242.670  28273.337249  28212.002751  28242.670   
1179471        28240.86      28243.785  28275.001879  28212.568121  28243.785   
1179472        28241.40      28245.790  28270.641630  28220.938370  28245.790   
1179473        28248.14      28246.790  28273.693401  28219.886599  28246.790   

               RSI       MACD  Signal_line  Historical_v

In [15]:
# フィボナッチリトレースメントを計算する関数
def calculate_fibonacci_retracements(high, low):
    retracement_levels = [0.0, 0.236, 0.382, 0.5, 0.618, 0.786, 1.0]
    retracements = {}
    
    for level in retracement_levels:
        retracements[level] = high - (high - low) * level

    return retracements

# データフレーム内の特定の範囲に対するリトレースメント水準を計算
range_start = 100  # 範囲の開始インデックス
range_end = 200    # 範囲の終了インデックス

high = df.loc[range_start:range_end, 'High'].max()
low = df.loc[range_start:range_end, 'Low'].min()

retracements = calculate_fibonacci_retracements(high, low)

# リトレースメント水準を表示
print("Fibonacci retracement levels:")
for level, value in retracements.items():
    print(f"{level * 100}%: {value}")

Fibonacci retracement levels:
0.0%: 28715.0
23.599999999999998%: 28623.55
38.2%: 28566.975
50.0%: 28521.25
61.8%: 28475.525
78.60000000000001%: 28410.425
100.0%: 28327.5


In [16]:
# ストキャスティクスを計算する関数
def calculate_stochastics(dataframe, high_col, low_col, close_col, k_window, d_window):
    highest_high = dataframe[high_col].rolling(window=k_window).max()
    lowest_low = dataframe[low_col].rolling(window=k_window).min()
    
    stoch_k = (dataframe[close_col] - lowest_low) / (highest_high - lowest_low) * 100
    stoch_d = stoch_k.rolling(window=d_window).mean()

    return stoch_k, stoch_d

# ストキャスティクスを計算して新たなカラムとして追加
k_window = 14  # %Kのウィンドウ期間（例：14分）
d_window = 3   # %Dのウィンドウ期間（例：3分）

df['%K'], df['%D'] = calculate_stochastics(df, 'High', 'Low', 'Close', k_window, d_window)

# 結果を表示
print(df.tail())

                   Timestamp     Open     High      Low    Close  Volume  \
1179469  2023-03-30 16:49:00  28235.2  28235.3  28228.0  28229.1  18.223   
1179470  2023-03-30 16:50:00  28229.1  28241.9  28229.1  28241.0  21.137   
1179471  2023-03-30 16:51:00  28241.0  28258.1  28241.0  28258.0  32.133   
1179472  2023-03-30 16:52:00  28258.0  28258.6  28243.7  28243.7  64.725   
1179473  2023-03-30 16:53:00  28243.7  28268.9  28243.7  28268.9  33.430   

         SMA_short_term  SMA_long_term      BB_upper      BB_lower  BB_middle  \
1179469        28238.04      28243.600  28275.168868  28212.031132  28243.600   
1179470        28236.38      28242.670  28273.337249  28212.002751  28242.670   
1179471        28240.86      28243.785  28275.001879  28212.568121  28243.785   
1179472        28241.40      28245.790  28270.641630  28220.938370  28245.790   
1179473        28248.14      28246.790  28273.693401  28219.886599  28246.790   

               RSI       MACD  Signal_line  Historical_v

In [17]:
# オンバランスボリュームを計算する関数
def calculate_obv(dataframe, close_col, volume_col):
    obv = [0]
    
    for i in range(1, len(dataframe)):
        if dataframe[close_col][i] > dataframe[close_col][i-1]:
            obv.append(obv[-1] + dataframe[volume_col][i])
        elif dataframe[close_col][i] < dataframe[close_col][i-1]:
            obv.append(obv[-1] - dataframe[volume_col][i])
        else:
            obv.append(obv[-1])
            
    return obv

# オンバランスボリュームを計算して新たなカラムとして追加
df['OBV'] = calculate_obv(df, 'Close', 'Volume')

# 結果を表示
print(df.tail())

                   Timestamp     Open     High      Low    Close  Volume  \
1179469  2023-03-30 16:49:00  28235.2  28235.3  28228.0  28229.1  18.223   
1179470  2023-03-30 16:50:00  28229.1  28241.9  28229.1  28241.0  21.137   
1179471  2023-03-30 16:51:00  28241.0  28258.1  28241.0  28258.0  32.133   
1179472  2023-03-30 16:52:00  28258.0  28258.6  28243.7  28243.7  64.725   
1179473  2023-03-30 16:53:00  28243.7  28268.9  28243.7  28268.9  33.430   

         SMA_short_term  SMA_long_term      BB_upper      BB_lower  BB_middle  \
1179469        28238.04      28243.600  28275.168868  28212.031132  28243.600   
1179470        28236.38      28242.670  28273.337249  28212.002751  28242.670   
1179471        28240.86      28243.785  28275.001879  28212.568121  28243.785   
1179472        28241.40      28245.790  28270.641630  28220.938370  28245.790   
1179473        28248.14      28246.790  28273.693401  28219.886599  28246.790   

               RSI       MACD  Signal_line  Historical_v

In [21]:
df.to_csv('bybit_btcusdt_20210101-20230330_features.csv')